# DAY 1: Acquiring and cleaning data (Python)

## Acquiring data from APIs

Hopefully you were able to attend yesterday's session on using APIs to acquire data, but here's a quick refresher on the dataset if not.

For the project we’ll pursue during DataFest, we’re going to access data stored on the Harvard Dataverse. A Dataverse is open source software for repositing research data. Once data is stored in a Dataverse, it can be accessed programmatically using the Dataverse API. We will use the Python package `dataverse` as an interface for the Dataverse API.

Here are three COVID-19 datasets from the Harvard Dataverse:

* [US data on COVID-19 cases and deaths, daily at state-level or county-level](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HIDLTK)
* [US data on COVID-19 cases and deaths, daily at metropolitan-level](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/5B8YM8)
* [World data on COVID-19 cases and deaths, daily at country-level](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/L20LOT)

We're going to use daily data on COVID-19 cases from the U.S. at the state-level (from dataset #1 above). These data span the period from January 21st 2020 until November 29th 2020 for each U.S. state (and the District of Columbia). If you wish, you may choose to use one of the other datasets for your project.

We can use the pyDataverse module as an interface for the API:

In [ ]:
# Install pyDataverse, if it isn't already there
!pip install pyDataverse

In [ ]:
# Import Dataverse modules
from pyDataverse.api import Api
from pyDataverse.models import Dataverse

# Import the usual data science suspects
import pandas as pd
import numpy as np

# BytesIO so we can load the API response into pandas
from io import BytesIO

In [ ]:
# get the digital object identifier for the Dataverse dataset
DOI = "doi:10.7910/DVN/HIDLTK"

# establish connection
base_url = 'https://dataverse.harvard.edu/'
api = Api(base_url)
print(api.status)

In [ ]:
covid = api.get_dataset(DOI)

In [ ]:
# Get a list of files, iterate through it, and show what's available
covid_files_list = covid.json()['data']['latestVersion']['files']

# view available files
for fileObject in covid_files_list:
    print("File name is {}; id is {}".format(fileObject["dataFile"]["filename"], fileObject["dataFile"]["id"]))

In [ ]:
# get data file for COVID-19 cases
US_states_cases_file = api.get_datafile("4201597")

## Read the data into pandas

Here ends the API recap and begins data cleanup. We'll use the Python library `pandas` to work with our data. The first thing we'll do is read it, but our code looks a bit different from what you'd probably use on your own data. Normally, this is what you'll do:

    df = pd.read_csv("some-file.csv")

Where `df` is the dataframe object you create and `some-file.csv` is the file you want to read, either as an absolute file path or one relative to the location you're running your notebook in.

In [ ]:
# Read the file into a pandas dataframe using BytesIO
US_states_cases = pd.read_csv(BytesIO(US_states_cases_file.content),sep='\t')

In [ ]:
# Take a look at the top of the dataframe
US_states_cases.head(10)

In [ ]:
# Take a look at the bottom of the dataframe
US_states_cases.tail(10)

# Cleaning data

## COVID-19 cases data

The COVID-19 cases data are in wide format, with individual columns for each day’s case counts. To visualize and analyze the data, it will be much easier to reshape the data so that it is organized in long format, with a single column for case counts and another column indicating the date those counts are associated with.

In addition, it will be useful to derive some time-related variables (e.g., day of year, week of year) from the dates. Finally, we should transform our cumulative case counts into regular counts and create some rate variables by normalizing by population count.

In [ ]:
# "melt" the dataframe into a tall format. Hover over `melt` in Google Colab to see a description of the function and parameters

US_states_cases_tall = pd.melt(US_states_cases,
        id_vars=['fips','NAME','POP70','HHD70','POP80','HHD80','POP90','HHD90','POP00','HHD00','POP10','HHD10'],
        var_name='date', value_name='cases_cumulative')

In [ ]:
US_states_cases_tall.head()

In [ ]:
US_states_cases_tall[US_states_cases_tall.NAME=='Massachusetts'].head(10)

In [ ]:
# Suppress scientific notation
# I figured this out by copying from https://stackoverflow.com/questions/17737300/suppressing-scientific-notation-in-pandas
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
US_states_cases_tall.cases_cumulative.describe()

Now that we've re-shaped the data, we'll want to add some new columns, derived from the data we have. The first thing we want to do is get the count of cases recorded on a given day, derived from the change in the cumulative count recorded.

To do that, we'll first sort by state and date, to put everything in order. We can sort by date even though the dates aren't stored as a date data type because this dataset uses [ISO 8601](https://www.iso.org/iso-8601-date-and-time-format.html), which is the [correct way to represent dates](https://xkcd.com/1179/).

Then we'll reset the index, since we no longer care how things were ordered before.

Last but not least, we'll group by state and apply a function using `shift` to get the value of the cumulative count in the previous row and subtract it from the current row's value.

Pay attention to all of the `inplace` keywords here. Pandas functions usually return a new dataframe rather than modifying one in place, and assigning a modified dataframe back to the same variable name can cause unexpected behavior, so modifying your data in place is often the way to go.

In [ ]:
US_states_cases_tall.sort_values(['fips', 'date'], inplace=True)

In [ ]:
US_states_cases_tall.reset_index(drop=True, inplace=True)

In [ ]:
# create cases counts
US_states_cases_tall['cases_count'] = US_states_cases_tall.groupby('NAME').cases_cumulative.apply(lambda x: x - x.shift(1)).fillna(0)
US_states_cases_tall.tail(10)

In [ ]:
# Are there any negative case counts?
US_states_cases_tall[US_states_cases_tall.cases_count < 0]

In [ ]:
# Are the negative case counts real?
US_states_cases_tall.loc[6818:6820]

In [ ]:
# tidy-up negative counts
US_states_cases_tall["cases_count_pos"] = np.where(US_states_cases_tall["cases_count"] < 0, 0, US_states_cases_tall["cases_count"])

Now we'll get some more information about the date

In [ ]:
# Get the day of the year (1-365)
US_states_cases_tall['day_of_year'] = pd.to_datetime(US_states_cases_tall.date).dt.dayofyear

In [ ]:
# Get day of week, where 0=Monday and 6=Sunday
US_states_cases_tall['day_of_week'] = pd.to_datetime(US_states_cases_tall.date).dt.weekday

Let's turn these days of week as numbers into words with a very simple example of the `apply` function. The way that days of the week are represented work well with list indices, so we can just make a lookup list and write a simple function to apply it. This function is equivalent to using

    lambda x: days[x]
    
as the function passed to the apply function, but I wanted to show this other approach.

In [ ]:
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

In [ ]:
def day_lookup(day_as_number):
    return days[day_as_number]

In [ ]:
US_states_cases_tall['day_of_week_words'] = US_states_cases_tall.day_of_week.apply(day_lookup)

In [ ]:
# Get week of year (1-52)
US_states_cases_tall["week_of_year"] = pd.to_datetime(US_states_cases_tall.date).dt.isocalendar().week

In [ ]:
# Get the month (1-12)
US_states_cases_tall["month"] = pd.to_datetime(US_states_cases_tall.date).dt.month

A frequent statistic in COVID news is the case rate per 100,000 people, as a way of normalizing for population. We'll get this statistic for both the daily rates and the cumulative rates for each state.

In [ ]:
US_states_cases_tall["cases_rate_100K"] = (US_states_cases_tall["cases_count_pos"] / US_states_cases_tall["POP10"]) * 1e5
US_states_cases_tall["cases_cumulative_rate_100K"] = (US_states_cases_tall["cases_cumulative"] / US_states_cases_tall["POP10"]) * 1e5

In [ ]:
# We can admire our work now
US_states_cases_tall.head()

In [ ]:
US_states_cases_tall.tail()

In [ ]:
# Now we can write out our data
US_states_cases_tall.to_csv('US_states_cases_tall.csv', index=None)